In [1]:
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd

In [2]:
cred = credentials.Certificate("service-account.json")
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
mapping = {
    'user':'users_dev',
    'booking':'bookings_dev',
    'emailReminders':'email_reminders_dev',
    'smsReminders':'sms_reminders_dev'
}

In [4]:
for key in mapping:
    print(mapping[key])

users_dev
bookings_dev
email_reminders_dev
sms_reminders_dev


In [5]:
def manipulate(x):
    temp =x.to_dict()
    temp['id']=x.id
    return temp

In [6]:
for key in mapping:
    temp = list(db.collection('{}'.format(key)).stream())
    temp_dict = list(map(manipulate, temp))
    globals()[f"df_{mapping[key]}"] = pd.DataFrame(temp_dict)
    globals()[f"df_{mapping[key]}"]['_imported_at'] = datetime.now()

In [7]:
df_email_reminders_dev.head()

,message,userId,id,_imported_at
0,"{'queued_reason': None, 'email': 'jason+007@na...",5329,00c353db-0a00-4bf6-a7a9-0099b1d645e1-5329,2023-04-11 12:12:49.242127
1,"{'queued_reason': None, 'email': 'jason+008@na...",5357,00c353db-0a00-4bf6-a7a9-0099b1d645e1-5357,2023-04-11 12:12:49.242127
2,"{'queued_reason': None, 'email': 'tatenda+cust...",5370,00c353db-0a00-4bf6-a7a9-0099b1d645e1-5370,2023-04-11 12:12:49.242127
3,"{'queued_reason': None, 'email': 'tatenda+cust...",5370,01b2cf25-bfa4-46bd-90c5-13ac7162c6cd-5370,2023-04-11 12:12:49.242127
4,"{'queued_reason': None, 'email': 'diana.rojas@...",5578,01b2cf25-bfa4-46bd-90c5-13ac7162c6cd-5578,2023-04-11 12:12:49.242127


In [8]:
key_path = r"C:\Users\nites\OneDrive\Documents\napo-nitesh-local-ae32-vpcservice-datawarehouse-879a160a28ee.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

job_config2 = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

In [9]:
dataset_name='firestore_dev'

for key in mapping:
    load_job = client.load_table_from_dataframe(
        globals()[f"df_{mapping[key]}"]
        ,"ae32-vpcservice-datawarehouse.{}.{}".format(dataset_name,mapping[key])
        ,job_config=job_config2
        ,location='EU'
    ) 
    print(load_job.result())

NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/ae32-vpcservice-datawarehouse/jobs?uploadType=multipart: Not found: Dataset ae32-vpcservice-datawarehouse:firestore_dev